# Playing around with convection

In [1]:
import underworld3 as uw
import sympy
from mpi4py import MPI  # library for displaying


import os
import numpy as np
import pyvista as pv
from mpi4py import MPI
import mpi4py


import numpy as np
import pyvista as pv
import vtk

In [2]:
mesh = uw.meshing.UnstructuredSimplexBox(minCoords=(0,0), maxCoords=(1,1), cellSize=0.1, qdegree=3)

In [3]:
v = uw.discretisation.MeshVariable("U", mesh, mesh.dim, degree=3)
p = uw.discretisation.MeshVariable("P", mesh, 1, degree=3)
T = uw.discretisation.MeshVariable("T", mesh, 1, degree=3)

In [4]:
## make the bodyforce
g=9.81 ## gravities acceleration
alpha = 0.1 ## coeffecient of thermal expansion
rho0 = 1000 ## background density
kappa = 0.001 ## thermal conductivity
heatingTemp = 100
viscosity = 1

stokes = uw.systems.Stokes(mesh, velocityField=v, pressureField=p)
stokes.constitutive_model = uw.systems.constitutive_models.ViscousFlowModel(mesh.dim)
stokes.constitutive_model.Parameters.viscosity = 1

## final thing sets the components that we actually set
stokes.add_dirichlet_bc( (0, 0), ["Top", "Bottom", "Left", "Right"], (0, 1) )





stokes.bodyforce = sympy.Matrix( [0, -g * rho0*(1 - alpha*T.sym[0]) ] )

adSolver = uw.systems.AdvDiffusion(mesh,u_Field=T, V_Field=v)
adSolver.constitutive_model = uw.systems.constitutive_models.DiffusionModel(mesh.dim)
adSolver.constitutive_model.Parameters.diffusivity = kappa
adSolver.theta = 0.5

x,y = mesh.X

adSolver.add_dirichlet_bc(heatingTemp*x, "Bottom")
adSolver.add_dirichlet_bc(-heatingTemp*(1-x), "Top")
adSolver.add_neumann_bc(0, "Left")
adSolver.add_neumann_bc(0, "Right")


Then we want to know what the Raylieigh number and Gr is here. Lets make both large to get some nice turbulent flows.


In [5]:
Ra = alpha*(2*heatingTemp) * g * 1**3 /(kappa*viscosity)
typicalVel = g * alpha*(2 * heatingTemp) * 1**2/viscosity
print(Ra, typicalVel)

196200.0 196.20000000000002


In [6]:
def plotVel(v, mesh, step, time):
    if mpi4py.MPI.COMM_WORLD.size == 1:

        
        pv.start_xvfb()
        pv.global_theme.background = "white"
        pv.global_theme.window_size = [750, 1200]
        pv.global_theme.antialiasing = True
        pv.global_theme.jupyter_backend = "panel"
        pv.global_theme.smooth_shading = True
        mesh.vtk("tmp_mesh.vtk")
        pvmesh = pv.read("tmp_mesh.vtk")
        ##pvmesh.point_data["P"] = uw.function.evaluate(p.sym[0], mesh.data)
        
        pvmesh.point_data["V"] = uw.function.evaluate(v.sym.dot(v.sym), mesh.data)
        arrow_loc = np.zeros((v.coords.shape[0], 3))
        arrow_loc[:, 0:2] = v.coords[...]
        arrow_length = np.zeros((v.coords.shape[0], 3))
        arrow_length[:, 0] = uw.function.evaluate(v.sym[0], v.coords)
        arrow_length[:, 1] = uw.function.evaluate(v.sym[1], v.coords)
        pl = pv.Plotter(window_size=[1000, 1000])
        pl.add_axes()
        pl.add_mesh(
            pvmesh,
            cmap="coolwarm",
            edge_color="Black",
            show_edges=True,
            scalars="V",
            use_transparency=False,
            opacity=1.0,
        )

        info = " steps: " + str(step) + " time: " + str(time)
        pl.add_text(info, position='upper_left', font_size=20, color='red')

        ## normalise the arrow length

        m = float('-inf')
        for el in arrow_length:
            norm = (el[0]**2 + el[1]**2)**0.5
            if (norm > m):
                m = norm


        maxScale = 0.01

        for index in range(len(arrow_length)):
            arrow_length[0] = arrow_length[0]/m * maxScale
            arrow_length[1] = arrow_length[1]/m * maxScale

        pl.add_arrows(arrow_loc, arrow_length, mag=3)
        ##pl.show(cpos="xy")

        stringName = "images/" + "velo"  + str(step) + ".png"
        pl.screenshot(stringName)
        
        

In [7]:
def plotTemp(s, mesh, step, time):
    ## s is a solver?


    # Ensure the output directory exists
    os.makedirs('images', exist_ok=True)

    with mesh.access():  # Access the mesh
        # Evaluate the mesh function
        mesh_numerical_soln = uw.function.evaluate(s.u.fn, mesh.data)

    if MPI.COMM_WORLD.size == 1:
        pv.global_theme.background = "white"
        pv.global_theme.window_size = [500, 500]
        pv.global_theme.antialiasing = True
        pv.global_theme.jupyter_backend = "panel"
        pv.global_theme.smooth_shading = True
        
        mesh_file = "ignore_mesh.vtk"
        
        # Ensure the mesh file is written
        try:
            mesh.vtk(mesh_file)
        except Exception as e:
            print(f"Error writing the mesh file: {e}")
            raise
        
        # Ensure the mesh file can be read
        try:
            pvmesh = pv.read(mesh_file)
        except Exception as e:
            print(f"Error reading the mesh file: {e}")
            raise
        
        pvmesh.point_data["T"] = mesh_numerical_soln
        sargs = dict(interactive=True)
        
        # Create the plot
        pl = pv.Plotter()
        

        
        # Add the mesh
        pl.add_mesh(
            pvmesh,
            cmap="coolwarm",
            edge_color="Black",
            show_edges=True,
            scalars="T",
            use_transparency=False,
            opacity=1,
            scalar_bar_args=sargs,
        )


        
        # Add the information text
        info = " steps: " + str(step) + " time: " + str(time)
        pl.add_text(info, position='upper_left', font_size=20, color='red')
        
        pl.camera_position = "xy"
        
        # Define output file name
        stringName = "images/" + str(step) + ".png"
        
        # Take a screenshot
        try:
            pl.screenshot(stringName)
        except Exception as e:
            print(f"Error taking a screenshot: {e}")
            raise

        






In [8]:
time = 0
for step in range(10000):
    if (step % 20 == 0):
        plotTemp(adSolver, mesh, step, time)
        plotVel(v, mesh, step, time)
    stokes.solve()
    dt = stokes.estimate_dt()
    time += dt
    adSolver.solve(timestep=dt)
    print("step", step)

    


/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2577: PyVistaDeprecationWarning: antialising is deprecated.  Please use `anti_aliasing` instead.
  warnings.warn(
/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2545: PyVistaDeprecationWarning: `anti_aliasing` is now a string or None and must be either "ssaa", "msaa", "fxaa", or None
  warnings.warn(


/var/folders/s9/rd03rq4x0px79z1jnyqlz3540000gn/T/ipykernel_98874/2767704975.py:16: PyVistaDeprecationWarning: `panel` backend is deprecated and is planned for future removal.
  pv.global_theme.jupyter_backend = "panel"
/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2577: PyVistaDeprecationWarning: antialising is deprecated.  Please use `anti_aliasing` instead.
  warnings.warn(
/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2545: PyVistaDeprecationWarning: `anti_aliasing` is now a string or None and must be either "ssaa", "msaa", "fxaa", or None
  warnings.warn(


/var/folders/s9/rd03rq4x0px79z1jnyqlz3540000gn/T/ipykernel_98874/2353216217.py:9: PyVistaDeprecationWarning: `panel` backend is deprecated and is planned for future removal.
  pv.global_theme.jupyter_backend = "panel"
/var/folders/s9/rd03rq4x0px79z1jnyqlz3540000gn/T/ipykernel_98874/2353216217.py:48: RuntimeWarning: invalid value encountered in divide
  arrow_length[0] = arrow_length[0]/m * maxScale
/var/folders/s9/rd03rq4x0px79z1jnyqlz3540000gn/T/ipykernel_98874/2353216217.py:49: RuntimeWarning: invalid value encountered in divide
  arrow_length[1] = arrow_length[1]/m * maxScale


  0 SNES Function norm 339.338 
  1 SNES Function norm 0.000789931 
Nonlinear Stokes_1_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1
  0 SNES Function norm 0.440837 
  1 SNES Function norm 3.16953e-06 
Nonlinear AdvDiff_slcn_1_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1
step 0
  0 SNES Function norm 505.105 
  1 SNES Function norm 0.00145942 
Nonlinear Stokes_1_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1
  0 SNES Function norm 106.925 
  1 SNES Function norm 0.00104288 
Nonlinear AdvDiff_slcn_1_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1
step 1
  0 SNES Function norm 584.32 
  1 SNES Function norm 0.000458937 
Nonlinear Stokes_1_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1
  0 SNES Function norm 160.549 
  1 SNES Function norm 0.00156379 
Nonlinear AdvDiff_slcn_1_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1
step 2
  0 SNES Function norm 591.101 
  1 SNES Function norm 0.000562282 
No

/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2577: PyVistaDeprecationWarning: antialising is deprecated.  Please use `anti_aliasing` instead.
  warnings.warn(
/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2545: PyVistaDeprecationWarning: `anti_aliasing` is now a string or None and must be either "ssaa", "msaa", "fxaa", or None
  warnings.warn(


...

  0 SNES Function norm 675.545 
  1 SNES Function norm 0.0181097 
Nonlinear Stokes_1_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1
  0 SNES Function norm 3043.57 
  1 SNES Function norm 0.0336175 
Nonlinear AdvDiff_slcn_1_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1
step 1320
  0 SNES Function norm 675.548 
  1 SNES Function norm 0.00761784 
Nonlinear Stokes_1_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1
  0 SNES Function norm 3043.69 
  1 SNES Function norm 0.0336188 
Nonlinear AdvDiff_slcn_1_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1
step 1321
  0 SNES Function norm 675.551 
  1 SNES Function norm 0.00358112 
Nonlinear Stokes_1_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1
  0 SNES Function norm 3043.69 
  1 SNES Function norm 0.0336188 
Nonlinear AdvDiff_slcn_1_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1
step 1322
  0 SNES Function norm 675.554 
  1 SNES Function norm 0.0181137 
No

/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2577: PyVistaDeprecationWarning: antialising is deprecated.  Please use `anti_aliasing` instead.
  warnings.warn(
/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2545: PyVistaDeprecationWarning: `anti_aliasing` is now a string or None and must be either "ssaa", "msaa", "fxaa", or None
  warnings.warn(


Error taking a screenshot: Render window is not current.


/var/folders/s9/rd03rq4x0px79z1jnyqlz3540000gn/T/ipykernel_98874/2767704975.py:16: PyVistaDeprecationWarning: `panel` backend is deprecated and is planned for future removal.
  pv.global_theme.jupyter_backend = "panel"


RenderWindowUnavailable: Render window is not current.